In [1]:
from lusidtools.jupyter_tools import toggle_code

# Computing P&L and Accrued interest for Bonds

In this notebook, we demonstrate how P&L and accrued interests can be computed for Bond instruments as well as how to book in bond coupons to our cash position. 
In this example, we will be using a 30Y US Treasury bond.

## Table of Contents:
- 1. [Create a portfolio](#1.-Create-Portfolio)
- 2. [Creating a bond instrument](#2.-Create-Bond)
- 3. [Transactions](#3.-Transactions)
- 4. [Quotes](#4.-Quotes)
- 5. [Valuation & PnL](#5.-Valuations)
- 6. [Cash flows](#6.-Cash-Flows)

In [2]:
# Import generic non-LUSID packages
import os
import pandas as pd
import numpy as np
from datetime import datetime
import json
import pytz
import time
from IPython.core.display import HTML
from utilities.formatting_tools import cashladder_to_df

# Import key modules from the LUSID package
import lusid as lu
import lusid.models as lm

# Import key functions from Lusid-Python-Tools and other packages
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.transaction_type_upload import upsert_transaction_type_alias
from lusidtools.lpt.lpt import to_date
from lusidjam import RefreshingToken


# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format
# display(HTML("<style>.container { width:90% !important; }</style>"))

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

# For running the notebook locally
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Authenticate our user and create our API client
api_factory = lu.utilities.ApiClientFactory(
    token=RefreshingToken(), api_secrets_filename=secrets_path
)

print("LUSID Environment Initialised")
print(
    "LUSID API Version :",
    api_factory.build(lu.api.ApplicationMetadataApi).get_lusid_versions().build_version,
)

LUSID Environment Initialised
LUSID API Version : 0.6.8203.0


In [3]:
# LUSID Variable Definitions
portfolio_api = api_factory.build(lu.api.PortfoliosApi)
transaction_portfolios_api = api_factory.build(lu.api.TransactionPortfoliosApi)
instruments_api = api_factory.build(lu.api.InstrumentsApi)
quotes_api = api_factory.build(lu.api.QuotesApi)
configuration_recipe_api = api_factory.build(lu.api.ConfigurationRecipeApi)
system_configuration_api = api_factory.build(lu.api.SystemConfigurationApi)
aggregration_api = api_factory.build(lu.api.AggregationApi)

In [4]:
# Define scopes
scope = "ibor"
quotes_scope = "ibor"

# 1. Create Portfolio

We must first create a portfolio to keep our bond in, this will allow us to perform a valuation and inspect the cash flows at a later stage.

In [5]:
portfolio_code = "BondPortfolioForPnLCalc"

try:
    transaction_portfolios_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=lm.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            code=portfolio_code,
            base_currency="USD",
            created="2010-01-01",
            sub_holding_keys=[],
        ),
    )

except lu.ApiException as e:
    print(json.loads(e.body)["title"])

Could not create a portfolio with id BondPortfolioForPnLCalc because it already exists in scope ibor.


# 2. Create Bond

We create a function that takes in our bond variables and upserts a bond to LUSID.

In [6]:
def create_bond(
    currency,
    payment_frequency,
    roll_convention,
    day_count_convention,
    payment_calendars,
    reset_calendars,
    settle_days,
    reset_days,
    start_date,
    maturity_date,
    dom_ccy,
    principal,
    coupon_rate,
    bond_identifier,
    bond_name,
):

    flow_conventions = lm.FlowConventions(
        currency=currency,
        payment_frequency=payment_frequency,
        roll_convention=roll_convention,
        day_count_convention=day_count_convention,
        payment_calendars=payment_calendars,
        reset_calendars=reset_calendars,
        settle_days=settle_days,
        reset_days=reset_days,
    )

    bond = lm.Bond(
        start_date=start_date,
        maturity_date=maturity_date,
        dom_ccy=dom_ccy,
        principal=principal,
        coupon_rate=coupon_rate,
        flow_conventions=flow_conventions,
        identifiers={},
        instrument_type="Bond",
        calculation_type="Standard",
    )

    # define the instrument to be upserted
    bond_definition = lm.InstrumentDefinition(
        name=bond_name,
        identifiers={"ClientInternal": lm.InstrumentIdValue(bond_identifier)},
        definition=bond,
    )

    # upsert the instrument
    upsert_request = {bond_identifier: bond_definition}
    upsert_response = instruments_api.upsert_instruments(request_body=upsert_request)
    bond_luid = upsert_response.values[bond_identifier].lusid_instrument_id
    print(bond_luid)

# 2.1 Upsert a bond

We can now set the variables of the bond and upsert it into LUSID using the function we built in the cell above.

In [7]:
currency = "USD"
payment_frequency = "6M"
roll_convention = "none"
day_count_convention = "ActualActual"
payment_calendars = []
reset_calendars = []
settle_days = 0
reset_days = 0
start_date = datetime(2019, 2, 15, 00, tzinfo=pytz.utc)
maturity_date = datetime(2049, 2, 15, 00, tzinfo=pytz.utc)
dom_ccy = "USD"
principal = 1
coupon_rate = 0.03
bond_identifier = "T_3_02/15/49"
bond_name = "T 3 02/15/49"

create_bond(
    currency,
    payment_frequency,
    roll_convention,
    day_count_convention,
    payment_calendars,
    reset_calendars,
    settle_days,
    reset_days,
    start_date,
    maturity_date,
    dom_ccy,
    principal,
    coupon_rate,
    bond_identifier,
    bond_name,
)

LUID_0000BYLX


# 3. Transactions

## 3.1 Create Transaction Request

Once the bond has been created, we can generate a transaction that adds it to our portfolio.

In [8]:
transactions = pd.read_csv("data/bond_transaction_data.csv")
transactions

,txn_id,type,client_id,trade_date,settlement_date,quantity,price,total_consideration,currency,portfolio
0,txn001,StockIn,T_3_02/15/49,2019-04-19T10:00:00Z,2019-04-20T10:00:00Z,1000000,100.76,1000000,USD,BondPortfolioForPnLCalc


In [9]:
for portfolio_code, grouped_df in transactions.groupby("portfolio"):

    transaction_request = [
        lm.TransactionRequest(
            transaction_id=row["txn_id"],
            type=row["type"],
            instrument_identifiers={
                "Instrument/default/ClientInternal": row["client_id"]
            },
            transaction_date=row["trade_date"],
            settlement_date=row["settlement_date"],
            units=row["quantity"],
            transaction_price=lm.TransactionPrice(price=row["price"], type="Price"),
            total_consideration=lm.CurrencyAndAmount(
                amount=row["total_consideration"], currency=row["currency"]
            ),
        )
        for index, row in grouped_df.iterrows()
    ]

    response = transaction_portfolios_api.upsert_transactions(
        scope=scope, code=portfolio_code, transaction_request=transaction_request
    )

    print(f"Transactions succesfully updated at time: {response.version.as_at_date}")

Transactions succesfully updated at time: 2021-11-15 15:01:34.316297+00:00


# 4. Quotes

The bond is in our portfolio, but currently lacks pricing. We will thus upsert quotes for the bond that we retrieved from our market data system. We upload prices from when the bond was created until today (at the time of writing, 29 October 2021). Below we can see the last 5 of the ~ 700 quotes that were uploaded.

In [10]:
bond_prices = pd.read_csv("data/bond_quotes_data.csv")
bond_prices

,client_internal,date,price,currency
0,T_3_02/15/49,22/04/2019,100.15,USD
1,T_3_02/15/49,23/04/2019,100.38,USD
2,T_3_02/15/49,14/08/2019,121.92,USD
3,T_3_02/15/49,15/08/2019,123.05,USD


In [11]:
# Create quotes request
instrument_quotes = {
    index: lm.UpsertQuoteRequest(
        quote_id=lm.QuoteId(
            quote_series_id=lm.QuoteSeriesId(
                provider="Lusid",
                instrument_id=row["client_internal"],
                instrument_id_type="ClientInternal",
                quote_type="Price",
                field="mid",
            ),
            effective_at=to_date(row["date"]),
        ),
        metric_value=lm.MetricValue(value=row["price"], unit=row["currency"]),
        scale_factor=100,
    )
    for index, row in bond_prices.iterrows()
}

# Upsert the quotes into LUSID
response = quotes_api.upsert_quotes(scope=scope, request_body=instrument_quotes)

if response.failed == {}:
    print(
        f"Quote successfully loaded into LUSID. {len(response.values)} quotes loaded."
    )

else:
    print(
        f"Some failures occurred during quotes upsertion, {len(response.failed)} did not get loaded into LUSID."
    )

Quote successfully loaded into LUSID. 4 quotes loaded.


# 5. Valuations

Once we have the bond booked into a portfolio, we can now value this portfolio. The recipe below describes how we will go about valuing the credit instruments in our portfolio (in this case just our bond). It details which model we will be using, SimpleStatic, which is supplied by Lusid.

## 5.1 Create valuation recipe

In [12]:
# Create a recipe to perform a valuation
configuration_recipe = lm.ConfigurationRecipe(
    scope=scope,
    code="bondValuation",
    market=lm.MarketContext(
        market_rules=[
            lm.MarketDataKeyRule(
                key="Credit.ClientInternal.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Price",
                field="mid",
                quote_interval="5D.0D",
            )
        ],
        options=lm.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="ClientInternal",
            default_scope=scope,
        ),
    ),
    pricing=lm.PricingContext(
        model_rules=[
            lm.VendorModelRule(
                supplier="Lusid",
                model_name="SimpleStatic",
                instrument_type="Bond",
                parameters="{}",
            )
        ]
    ),
)

upsert_configuration_recipe_response = (
    configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=lm.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )
)

## 5.2 Create daily valuation function

Once we have made a recipe, we can now create a function that outputs a dataframe with the valuation of our portfolio. We choose to display PV, Accrued Interest and PnL as well as some instrument identifiers.

In [13]:
def get_val(date, portfolio_code):

    valuation_request = lm.ValuationRequest(
        recipe_id=lm.ResourceId(scope=scope, code="bondValuation"),
        metrics=[
            lm.AggregateSpec("Instrument/default/Name", "Value"),
            lm.AggregateSpec("Instrument/default/ClientInternal", "Value"),
            lm.AggregateSpec("Quotes/Price", "Value"),
            lm.AggregateSpec("Holding/default/Units", "Value"),
            lm.AggregateSpec("Valuation/PV/Amount", "Value"),
            lm.AggregateSpec("Holding/default/Accrual", "Value"),
            lm.AggregateSpec("Valuation/PnL/Tm1", "Value"),
        ],
        group_by=["Instrument/default/Name"],
        portfolio_entity_ids=[lm.PortfolioEntityId(scope=scope, code=portfolio_code)],
        valuation_schedule=lm.ValuationSchedule(effective_at=date),
    )

    val_data = aggregration_api.get_valuation(valuation_request=valuation_request).data

    vals_df = pd.DataFrame(val_data)

    vals_df.rename(
        columns={
            "Instrument/default/Name": "InstrumentName",
            "Instrument/default/ClientInternal": "ClientInternal",
            "Valuation/PV/Amount": "Present Value",
            "Valuation/PnL/Tm1": "PnL (1-day)",
            "Holding/default/Accrual": "Accrued Interest",
        },
        inplace=True,
    )
    try:
        return vals_df.drop("Aggregation/Errors", axis=1)
    except:
        return vals_df

We will now display the first few days in the lifecycle of our bond. Recall that we have bought this bond on Friday 19 April 2019 and this bond will only settle by Monday 22 April. Day 1 in our scenario is thus 22 April 2019.

## Day 1

At 10AM on day 1, once our bond has settled, we value our bond position with the SimpleStatic model and find that it is worth USD 1,006,909.03. This value includes about two months worth of accrued interest as the last coupon was on 2019-02-15 and we are now valuing it on 2019-04-22.

In [14]:
get_val("2019-04-22T10:00:00Z", portfolio_code)

,Present Value,InstrumentName,ClientInternal,Quotes/Price,Holding/default/Units,Accrued Interest,PnL (1-day)
0,"1,006,909.03",T 3 02/15/49,T_3_02/15/49,100.15,"1,000,000.00","5,424.66",None


## Day 2

On day 2, we run our valuation again and find that our accrued interest increased and the price has moved up. Our position is now worth USD 2,347.82 more than on day 1. Our PnL after 1 day is thus USD 2,347.82 on this position.

In [15]:
get_val("2019-04-23T10:00:00Z", portfolio_code)

,Present Value,InstrumentName,ClientInternal,Quotes/Price,Holding/default/Units,Accrued Interest,PnL (1-day)
0,"1,009,256.85",T 3 02/15/49,T_3_02/15/49,100.38,"1,000,000.00","5,506.85","2,347.82"


## Coupon date

One day before our first coupon date, we see the accrued interest approaching the coupon value of USD 15,000. When the coupon date hits, the accrued interest will reset to zero. In our case, we want to add a cash position of about USD 15,000 to our portfolio so that we end up with the correct value on the coupon date. This is where our cash flow management functions come in.

In [16]:
get_val("2019-08-14T10:00:00Z", portfolio_code)

,Present Value,InstrumentName,ClientInternal,Quotes/Price,Holding/default/Units,Accrued Interest,PnL (1-day)
0,"1,234,013.27",T 3 02/15/49,T_3_02/15/49,121.92,"1,000,000.00","14,794.52","35,707.19"


In [17]:
get_val("2019-08-15T10:00:00Z", portfolio_code)

,Present Value,InstrumentName,ClientInternal,Quotes/Price,Holding/default/Units,Accrued Interest,PnL (1-day)
0,"1,230,468.75",T 3 02/15/49,T_3_02/15/49,123.05,"1,000,000.00",0.00,"-3,544.52"
1,"15,000.00",USD,None,nan,"15,000.00",0.00,0.00


# 6. Cash Flows

## 6.1 Cash Ladder

If we take a look at the cash ladder between our first day of owning this bond, 22 April 2019, and today (28 October 2021 at time of writing) we can see the coupons that will have been paid out to us during this timeframe.

In [18]:
cash_ladder = transaction_portfolios_api.get_portfolio_cash_ladder(
    scope=scope,
    code=portfolio_code,
    effective_at="2019-04-22T10:00:00Z",
    from_effective_at="2019-04-22T10:00:00Z",
    to_effective_at="2021-10-28T10:00:00Z",
)

cashladder_to_df(cash_ladder)

,Currency,Date,Activity,Value
0,USD,2019-08-15 00:00:00+00:00,Open,0.00
1,USD,2019-08-15 00:00:00+00:00,Coupon,"15,000.00"
2,USD,2019-08-15 00:00:00+00:00,Close,"15,000.00"
3,USD,2020-02-15 00:00:00+00:00,Open,"15,000.00"
4,USD,2020-02-15 00:00:00+00:00,Coupon,"15,000.00"
5,USD,2020-02-15 00:00:00+00:00,Close,"30,000.00"
6,USD,2020-08-15 00:00:00+00:00,Open,"30,000.00"
7,USD,2020-08-15 00:00:00+00:00,Coupon,"15,000.00"
8,USD,2020-08-15 00:00:00+00:00,Close,"45,000.00"
9,USD,2021-02-15 00:00:00+00:00,Open,"45,000.00"


We can also take a look and see all the future cash flows we can expect from this bond by running the same function, but setting the time frame from now until a few years in the future. In this example we are retrieving the expected cash flows from 2021-10-28 until 2025-10-28.

In [19]:
cash_ladder = transaction_portfolios_api.get_portfolio_cash_ladder(
    scope=scope,
    code=portfolio_code,
    effective_at="2021-10-28T10:00:00Z",
    from_effective_at="2021-10-28T10:00:00Z",
    to_effective_at="2025-10-28T10:00:00Z",
)
cashladder_to_df(cash_ladder)

,Currency,Date,Activity,Value
0,USD,2022-02-15 00:00:00+00:00,Open,"15,000.00"
1,USD,2022-02-15 00:00:00+00:00,Coupon,"15,000.00"
2,USD,2022-02-15 00:00:00+00:00,Close,"30,000.00"
3,USD,2022-08-15 00:00:00+00:00,Open,"30,000.00"
4,USD,2022-08-15 00:00:00+00:00,Coupon,"15,000.00"
5,USD,2022-08-15 00:00:00+00:00,Close,"45,000.00"
6,USD,2023-02-15 00:00:00+00:00,Open,"45,000.00"
7,USD,2023-02-15 00:00:00+00:00,Coupon,"15,000.00"
8,USD,2023-02-15 00:00:00+00:00,Close,"60,000.00"
9,USD,2023-08-15 00:00:00+00:00,Open,"60,000.00"


## 6.2 Cash Flow table

Below we can see the end of the cash flow table for our portfolio which contains our bond. We can see the coupons we are owed and the final principal repayment on the maturity date.

The table below is just to illustrate the final few payments and how the final payment with the principal at maturity date will be handled. 

In [20]:
upsertable_cash_flows = transaction_portfolios_api.get_upsertable_portfolio_cash_flows(
    scope=scope,
    code=portfolio_code,
    effective_at="2021-08-14T10:00:00Z",
    window_start="2045-02-15T10:00:00Z",
    window_end="2049-02-15T10:00:00Z",
)

# we create a dataframe out of the cash flows table and drop some columns to improve readability
cash_flow_table = lusid_response_to_data_frame(upsertable_cash_flows)
cash_flow_table.drop(
    [
        "instrument_identifiers",
        "instrument_uid",
        "properties",
        "source",
        "entry_date_time",
        "transaction_currency",
        "units",
        "transaction_price.price",
        "transaction_price.type",
        "exchange_rate",
    ],
    axis=1,
)

,transaction_id,type,transaction_date,settlement_date,total_consideration.amount,total_consideration.currency
0,-LUID_0000BYLX-USD-Receive,CashFlow,2045-08-15 00:00:00+00:00,2045-08-15 00:00:00+00:00,"15,000.00",USD
1,-LUID_0000BYLX-USD-Receive,CashFlow,2046-02-15 00:00:00+00:00,2046-02-15 00:00:00+00:00,"15,000.00",USD
2,-LUID_0000BYLX-USD-Receive,CashFlow,2046-08-15 00:00:00+00:00,2046-08-15 00:00:00+00:00,"15,000.00",USD
3,-LUID_0000BYLX-USD-Receive,CashFlow,2047-02-15 00:00:00+00:00,2047-02-15 00:00:00+00:00,"15,000.00",USD
4,-LUID_0000BYLX-USD-Receive,CashFlow,2047-08-15 00:00:00+00:00,2047-08-15 00:00:00+00:00,"15,000.00",USD
5,-LUID_0000BYLX-USD-Receive,CashFlow,2048-02-15 00:00:00+00:00,2048-02-15 00:00:00+00:00,"15,000.00",USD
6,-LUID_0000BYLX-USD-Receive,CashFlow,2048-08-15 00:00:00+00:00,2048-08-15 00:00:00+00:00,"15,000.00",USD
7,-LUID_0000BYLX-USD-Receive,CashFlow,2049-02-15 00:00:00+00:00,2049-02-15 00:00:00+00:00,"15,000.00",USD
8,-LUID_0000BYLX-USD-Receive,CashFlow,2049-02-15 00:00:00+00:00,2049-02-15 00:00:00+00:00,"1,000,000.00",USD


In our valuation of the coupon date we did in part "5.2 Create Daily Valuation Function", we saw the accrued interest for the coupon on 2019-08-15 reset. Now we will retrieve this cash flow and upsert it to LUSID so that in our valuation, we can see the bond and the cash position together.

In [21]:
get_val("2019-08-15T10:00:00Z", portfolio_code)

,Present Value,InstrumentName,ClientInternal,Quotes/Price,Holding/default/Units,Accrued Interest,PnL (1-day)
0,"1,230,468.75",T 3 02/15/49,T_3_02/15/49,123.05,"1,000,000.00",0.00,"-3,544.52"
1,"15,000.00",USD,None,nan,"15,000.00",0.00,0.00


In [22]:
upsertable_cash_flows = transaction_portfolios_api.get_upsertable_portfolio_cash_flows(
    scope=scope,
    code=portfolio_code,
    effective_at="2019-08-14T10:00:00Z",
    window_start="2019-08-14T10:00:00Z",
    window_end="2019-08-30T10:00:00Z",
)
lusid_response_to_data_frame(upsertable_cash_flows)

,transaction_id,type,instrument_identifiers,instrument_uid,transaction_date,settlement_date,units,transaction_price.price,transaction_price.type,total_consideration.amount,total_consideration.currency,exchange_rate,transaction_currency,properties,source,entry_date_time
0,-LUID_0000BYLX-USD-Receive,CashFlow,{},LUID_0000BYLX,2019-08-15 00:00:00+00:00,2019-08-15 00:00:00+00:00,"15,000.00",1.00,Price,"15,000.00",USD,1.00,USD,{},,0001-01-01 00:00:00+00:00


We want to book this cashflow into LUSID now so that we have an accurate cash position for our valuation. We have to make a few minor adjustments such as specifying which instrument we want to book these cash flows into. In our case, this is the USD currency identifier. We must also ensure the transaction ID's are unique, thus we tag on the date of the coupon to the transaction ID.

Once this is done, we upsert these cash flows to our portfolio.

In [23]:
for x in upsertable_cash_flows.values:
    x.instrument_identifiers = {"Instrument/default/Currency": "USD"}
    x.transaction_id = x.transaction_id + "-" + str(x.transaction_date)

In [24]:
upsert_transactions = transaction_portfolios_api.upsert_transactions(
    scope=scope, code=portfolio_code, transaction_request=upsertable_cash_flows.values
)

If we now want to see the valuation of our portfolio on the coupon date, we can see the bond position has diminished by the coupon amount and the cash position has increased by that amount.

In [25]:
get_val("2019-08-15T10:00:00Z", portfolio_code)

,Present Value,InstrumentName,ClientInternal,Quotes/Price,Holding/default/Units,Accrued Interest,PnL (1-day)
0,"1,230,468.75",T 3 02/15/49,T_3_02/15/49,123.05,"1,000,000.00",0.00,"-3,544.52"
1,"15,000.00",USD,None,nan,"15,000.00",0.00,0.00
